In [1]:
#some basic imports and setups
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt


#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'memes')
#image_dir = current_dir

%matplotlib inline

In [ ]:
#get list of all images
img_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('ting.jpg')]
print(len(img_files))
#load all images
imgs = []
for f in img_files:
    img = Image.open(f)
    img.thumbnail((227, 227), Image.ANTIALIAS)
    #img = img.resize((227,227))
    assert np.shape(img) == (227, 227, 3)
    imgs.append(img)
#use img.thumbnail for square images, img.resize for non square
    
#plot images
fig = plt.figure(figsize=(15,6))
for i, img in enumerate(imgs):
    fig.add_subplot(1,len(imgs),i+1)
    plt.imshow(img)

In [3]:
from alexnet import AlexNet
from caffe_classes import class_names

#placeholder for input and dropout rate
x = tf.placeholder(tf.float32, [1, 227, 227, 3])
keep_prob = tf.placeholder(tf.float32)

#create model with default config ( == no skip_layer and 1000 units in the last layer)
model = AlexNet(x, keep_prob, 1000,[],['fc7','fc8'],512) #maybe need to put fc8 in skip_layers

#define activation of last layer as score
score = model.fc6

#create op to calculate softmax 
#softmax = tf.nn.softmax(score)

In [ ]:
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    
# Create figure handle
    fig2 = plt.figure(figsize=(15,6))
    
    # Loop over all images
    for i, image in enumerate(imgs):
        
        # Convert image to float32 and resize to (227x227)
        #img = cv2.resize(image.astype(np.float32), (227,227))
        
        # Subtract the ImageNet mean
        img = image - imagenet_mean
        
        # Reshape as needed to feed into model
        img = img.reshape((1,227,227,3))
        
        # Run the session and calculate the class probability
        #probs = sess.run(softmax, feed_dict={x: img, keep_prob: 1})
        probs = sess.run(score, feed_dict={x: img, keep_prob: 1})
        
        # Get the class name of the class with the highest probability
        #class_name = class_names[np.argmax(probs)]
        
        
        # Plot image with class name and prob in the title
        fig2.add_subplot(len(imgs),1,i+1)
        plt.imshow(image)
        #plt.title("Class: " + class_name + ", probability: %.4f" %probs[0,np.argmax(probs)])
        plt.axis('off')
        plt.title("Vector: " + "%.4f" %probs[0,0])
        

In [ ]:
probs[0,:500]

In [ ]:
ind = np.argsort(probs)
for i in range(990,1000):
    print(class_names[int(ind[0][i])])
    print(probs[0,ind[0][i]])

In [ ]:
print(probs[0,:50])

Converting captions and meme vector representations into single Tfrecord

Requires putting memes through alexnet to find their vector rep, shuffling the captions, changing captions into their word2idx, finally saving one caption together with one meme.

In [ ]:
with open('captions.txt','r') as f:
    captions = f.readlines()
    

In [5]:
current_dir

'/Users/ALP/Desktop/Stanford/CS224n/MemeProject'

In [ ]:
len(captions)

In [ ]:
captions = list(set(captions))


In [ ]:
len(captions)

In [ ]:
captions = [s.lower() for s in captions]

In [ ]:
img_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('jpg')]
print(len(img_files))
img_files = list(set(img_files))
print(len(img_files))

In [ ]:
meme_name = img_files[2500].replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')
meme_name = meme_name.replace('.jpg','')
meme_name = meme_name.replace('-',' ').lower()
meme_name = 'dolan'
print(meme_name)
match = [s for s in captions if meme_name in s]
#print(match)
#match[0].replace(meme_name + ' - ', '')

In [6]:
search_dir = '/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes'
os.chdir(search_dir)
img_files = filter(os.path.isfile, os.listdir(search_dir))
img_files = [os.path.join(search_dir, f) for f in img_files] # add path to each file
img_files.sort(key=lambda x: os.path.getmtime(x))
with open('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
data_memes = []
data_captions = []
counter = 0

#Doing everything in one script: (the fc6 vectors are quite sparse)
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Load the pretrained weights into the model
    model.load_initial_weights(sess)
    
    for i,meme in enumerate(img_files):
        #meme_name = meme.replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')
        #meme_name = meme_name.replace('.jpg','').lower()
        #meme_name = meme_name.replace('-',' ')
        img = Image.open(meme)
        try:
            img.thumbnail((227, 227), Image.ANTIALIAS)
            #img = img.resize((227,227))
            #use img.thumbnail for square images, img.resize for non square
            assert np.shape(img) == (227, 227, 3)
        except AssertionError:
            img = img.resize((227,227))
            print('sizing error')
        
        # Subtract the ImageNet mean
        img = img - imagenet_mean #should probably change this
        
        # Reshape as needed to feed into model
        img = img.reshape((1,227,227,3))

        meme_vector = sess.run(score, feed_dict={x: img, keep_prob: 1}) #[1,4096]
        meme_vector = np.reshape(meme_vector,[4096])
        assert np.shape(meme_vector) == (4096,)
        #match = [s.split('-',1)[-1].lstrip() for s in captions if meme_name in s]
        match = []
        meme_name = captions[counter].split('-')[0]
        while meme_name in captions[counter]:
            match.append(captions[counter].split('-')[-1])
            counter += 1
        
        #now save in tfrecords format, or prepare for that action
        meme_vectors = [meme_vector for cap in match]
        assert len(meme_vectors) == len(match)
        data_memes.extend(meme_vectors)
        data_captions.extend(match)

        if i % 100 == 0:
            print(i,len(data_memes),len(data_captions))
        
 

IOError: [Errno 2] No such file or directory: 'bvlc_alexnet.npy'

In [ ]:
search_dir = '/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes'
os.chdir(search_dir)
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files] # add path to each file
files.sort(key=lambda x: os.path.getmtime(x))
print(files[:100])

In [ ]:
with open('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/Captions.txt','r') as f:
    captions = f.readlines()
#captions = list(set(captions))
captions = [s.lower() for s in captions]
print(len([s for s in captions if 'scared bekett' in s]))
captions[112000:112100]

In [8]:
del img_files[0]

In [11]:
img_files[2]

'/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/the-most-interesting-man-in-the-world.jpg'

In [12]:
for i,meme in enumerate(img_files):
    img_files[i] = meme.replace('/Users/ALP/Desktop/Stanford/CS224n/MemeProject/memes/','')

In [17]:
img_files[2503]

'mendozameme.jpg'

In [21]:
img_files[10]

'winter-is-coming.jpg'

In [22]:
f = open('ordered_memes.txt', 'w')
for item in img_files:
    f.write('%s\n' % item)

In [ ]:
deleters = []
for i,ting in enumerate(data_captions):
    if ting == '':
        deleters.append(i)
for i,ting in enumerate(deleters):
    del data_captions[ting-i]
    del data_memes[ting-i]

In [ ]:
import re
word_captions = []
for capt in data_captions:
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    word_captions.append(words)
#print(len(word_captions))
#word_captions = list(set(word_captions))
#print(len(word_captions))

In [ ]:
from collections import Counter
print("Creating vocabulary.")
counter = Counter()
for c in word_captions:
    counter.update(c)
print("Total words:", len(counter))

# Filter uncommon words and sort by descending count.
word_counts = [x for x in counter.items() if x[1] >= 3]
word_counts.sort(key=lambda x: x[1], reverse=True)
print("Words in vocabulary:", len(word_counts))

In [ ]:
# Create the vocabulary dictionary.
reverse_vocab = [x[0] for x in word_counts]
#unk_id = len(reverse_vocab)
vocab_dict = dict([(x, y) for (y, x) in enumerate(reverse_vocab)])

In [ ]:
reverse_vocab[1]

In [ ]:
EMBEDDING_DIMENSION=300 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = '~/Desktop/Stanford/CS224n/MemeProject'

PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers
weights = []
index_counter = 0

with open('glove.42B.300d.txt','r') as file:
    for index, line in enumerate(file):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        if word in vocab_dict:
            index_counter += 1
            word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
            word2idx[word] = index_counter # PAD is our zeroth index so shift by one
            weights.append(word_weights)
        if index % 20000 == 0:
            print(index)
        if index + 1 == 1500000:
            # Limit vocabulary to top 40k terms
            break

EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random #maybe include start and end token here
UNKNOWN_TOKEN=len(weights)
word2idx['UNK'] = UNKNOWN_TOKEN
word2idx['<S>'] = UNKNOWN_TOKEN + 1
word2idx['</S>'] = UNKNOWN_TOKEN + 2
weights.append(np.random.randn(EMBEDDING_DIMENSION))
weights.append(np.random.randn(EMBEDDING_DIMENSION))
weights.append(np.random.randn(EMBEDDING_DIMENSION))

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]

#Save Vocabulary
with tf.gfile.FastGFile('vocab.txt', "w") as f:
    f.write("\n".join(["%s %d" % (w, c) for w, c in word2idx.iteritems()]))
print("Wrote vocabulary file:", 'vocab.txt')

In [ ]:
with open('vocab.txt','r') as f:
    reverse_vocab = list(f.readlines())
reverse_vocab = [(line.split()[0],line.split()[1]) for line in reverse_vocab]

vocab = dict([(x, y) for (x, y) in reverse_vocab]) 

In [ ]:
print(vocab['.'])
x = sorted(vocab.iteritems(), key=lambda x: int(x[1]))
reverse_vocab = [y[0] for y in x]
print(reverse_vocab[44430:])

In [ ]:
filenames = [os.path.join(image_dir, f) for f in ['one_does_not_simply.jpg']]

In [ ]:
filenames

In [ ]:
weights[76984]

In [ ]:
np.savetxt('embedding_matrix2',weights)

In [ ]:
deleters = []
for i,ting in enumerate(data_captions):
    if len(ting) == 2:
        deleters.append(i)

In [ ]:
for i,ting in enumerate(deleters):
    del data_captions[ting-i]
    del data_memes[ting-i]

In [ ]:
deleters[0]

In [ ]:
len(data_captions)

In [ ]:
import re
token_captions = []
for capt in data_captions:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_caption.append(word2idx['</S>'])
    token_captions.append(token_caption)

In [ ]:
from __future__ import division
total_words = 0
total_UNK = 0
for i,ting in enumerate(token_captions):
    for word in ting:
        total_words += 1
        if word == word2idx['UNK']:
            total_UNK += 1
print(total_words - 2*len(data_captions))
print(total_UNK)
print((total_UNK/(total_words - 2*len(data_captions))))

In [ ]:
for i,ting in enumerate(deleters):
    del data_captions[ting-i]
    del data_memes[ting-i]
    del token_captions[ting-i]

In [ ]:
from random import shuffle
c = list(zip(data_memes, token_captions))
shuffle(c)
memes_shuffled, captions_shuffled = zip(*c)

In [ ]:
len(captions_shuffled)

In [ ]:

def _int64_feature(value):
    """Wrapper for inserting an int64 Feature into a SequenceExample proto."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    """Wrapper for inserting a bytes Feature into a SequenceExample proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature_list(values):
    """Wrapper for inserting an int64 FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])


def _bytes_feature_list(values):
    """Wrapper for inserting a bytes FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_bytes_feature(v) for v in values])

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [ ]:
memes_shuffled_int = []
for i,meme in enumerate(memes_shuffled):
    memes_shuffled_int.append(np.int_(meme*1000000000))
print(memes_shuffled_int[0][:100])

In [ ]:
(captions_shuffled[8])

In [ ]:
def _to_sequence_example(image, decoder, vocab):
    """Builds a SequenceExample proto for an image-caption pair.
    Args:
        image: An ImageMetadata object.
        decoder: An ImageDecoder object.
        vocab: A Vocabulary object.
      Returns:
        A SequenceExample proto.
      """
    with tf.gfile.FastGFile(image.filename, "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        return
    
    context = tf.train.Features(feature={
          "image/image_id": _int64_feature(image.image_id),
          "image/data": _bytes_feature(encoded_image),
      })
    
    assert len(image.captions) == 1
    caption = image.captions[0]
    caption_ids = [vocab.word_to_id(word) for word in caption]
    feature_lists = tf.train.FeatureLists(feature_list={
          "image/caption": _bytes_feature_list(caption),
          "image/caption_ids": _int64_feature_list(caption_ids)
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    return sequence_example

In [ ]:
import sys
train_filename = 'train.tfrecords4'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled_int)):
    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled_int))
        sys.stdout.flush()
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(memes_shuffled_int[i].tostring()),  #this is the part that needs to be a float save
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(captions_shuffled[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()